# Ollama access exeriments

Anton Antonov  
January 2026 

----

## Introduction

This notebook has ad hoc experiments accessing [Ollama](https://ollama.com) via "HTT::Tiny" and/or "cUrl".

For the API see https://github.com/ollama/ollama/blob/main/docs/api.md

---

## Setup

In [ ]:
use HTTP::Tiny
my $base-url = 'http://localhost:11434';

---

## Ping

In [ ]:
my $response = HTTP::Tiny.new.get("$base-url");

say (:$response);
say $response<content>.decode;

In [ ]:
my sub tiny-get(Str:D $path, :%headers = {}) {
    my $response = HTTP::Tiny.new.get($path);
    my %res = $response;
    if $response<success> {
        my $json-string = $response<content>.decode;
        %res<decoded-content> = $json-string;
    }
    return %res;
}

In [ ]:
tiny-get($base-url)

---

## Get models

In [ ]:
my $response = HTTP::Tiny.new.get("$base-url/api/tags");

say (:$response);
say $response<content>.decode;

my %res;
if $response<success> {
    my $json-string = $response<content>.decode;
    %res = from-json($json-string);

    say "Successfully parsed data:";
    %res; 
} else {
    say "Failed to fetch JSON: $response<reason>";
}

In [ ]:
#% html
%res<models>
==> { .sort(*<modified_at>.DateTime ) }()
==> { .map({ $_<modified_at> = $_<modified_at>.DateTime.Str.split('T').head; $_ }) }()
==> { .map({ $_<size-short> = round($_<size> / 1024 ** 3, 0.1) ~ ' GB'; $_ }) }()
==> to-html(field-names => <name size size-short modified_at>)

----

## Text generations

```shell
curl http://localhost:11434/api/generate -d '{
  "model": "llama3.1",
  "prompt": "Why is the sky blue?",
  "stream": false
}'
```

In [ ]:
my %body = 
    model => 'gemma3:1b',
    prompt => "How many people live in Brazil?",
    :!stream
;

my $response = HTTP::Tiny.post(
    "$base-url/api/generate",
    :headers{ 'Content-Type' => 'application/json' },
    content => to-json(%body)
);

my $res;
if $response<success> {
    say "POST successful!";
    $res = $response<content>.decode;
} else {
    say "POST failed: $response<reason>";
}

$res

In [ ]:
#% markdown
$res.lines.map({ from-json($_)<response> }).join

---

## Chat completions

```shell
curl http://localhost:11434/api/chat -d '{
  "model": "llama3.1",
  "messages": [
    {"role": "user", "content": "why is the sky blue?"}
  ]
}'
```

In [ ]:
to-json(%body, :!pretty)

In [ ]:
my %body =
    model => 'gemma3:1b',
    messages => [{role => "user", content => "How many people live in Brazil?"},]
;

my $response = HTTP::Tiny.post(
    "$base-url/api/chat",
    :headers{ 'Content-Type' => 'application/json' },
    content => to-json(%body)
);

my $res;
if $response<success> {
    say "POST successful!";
    $res = $response<content>.decode;
} else {
    say "POST failed: $response<reason>";
}

$res


In [ ]:
$res.lines.map({ from-json($_)<message><content> }).join